# Simple RAG

In [1]:
import os
import time
from dotenv import load_dotenv
from openai import OpenAI

from rag_utils import SearchEngine

In [2]:
class RAGChatbot:
    """
    A chatbot that uses a Retrieval-Augmented Generation (RAG) approach to answer questions.
    
    Attributes:
        se (SearchEngine): The search engine used to retrieve documents.
        enable_logging (bool): Flag to show progress messages.
        llm (OpenAI): The language model instance.
        llm_model (str): The name of the language model.
    """
    
    def __init__(self, search_engine: SearchEngine,hybrid_search = False, enable_logging=True):
        """
        Initializes the RAGChatbot with a search engine and optional progress display.
        
        Args:
            search_engine (SearchEngine): The search engine to use for document retrieval.
            enable_logging (bool): Whether to show progress messages. Default is True.
        """
        self.se = search_engine
        self.enable_logging = enable_logging
        self.hybrid_search = hybrid_search
        self.llm, self.llm_model = self.init_llm()
        if self.enable_logging:
            print("\nConnected to Search Engine and Language Model")
        self.messages = []
        
    def clear_history(self):
        """
        Clears the chat history.
        """
        self.messages = []
        
    def init_llm(self):
        """
        Initializes the language model using environment variables for API key and base URL.
        
        Returns:
            tuple: A tuple containing the language model instance and the model name.
        
        Raises:
            ValueError: If the API key is not set in environment variables.
        """
        load_dotenv()
        api_key = os.environ.get("OPENAI_API_KEY")
        base_url = os.environ.get("OPENAI_BASE_URL")
        
        if not api_key:
            raise ValueError("API key must be set in environment variables.")
        
        llm = OpenAI(api_key=api_key, base_url=base_url) if base_url else OpenAI(api_key=api_key)
        
        llm_model = os.environ.get("OPENAI_MODEL_NAME", "gpt-4o-mini")
        
        return llm, llm_model

    def answer(self, question, reference_num=2,temperature = 0.1,max_tokens = 1024, history_length = 6):
        """
        Answers a question using the RAG approach by retrieving documents and generating a response.
        
        Args:
            question (str): The question to answer.
            reference_num (int): The number of documents to retrieve. Default is 2.
            temperature (float): The sampling temperature for the language model. Default is 0.1.
            max_tokens (int): The maximum number of tokens to generate. Default is 1024.
        
        Returns:
            tuple: A tuple containing the generated answer and the retrieved documents.
        """
        start_search_time = time.time()
        if self.hybrid_search:
            result = self.se.hybrid_search(question, text_results=reference_num,semantic_results=reference_num)
        else:
            result = self.se.search(question, n_results=reference_num)
        documents = result['documents'][0]
        documents_path = [x['file'] for x in  result['metadatas'][0]]
        documents_res = [{'document':document,'path':document_path} for document,document_path in zip(documents,documents_path)]
        end_search_time = time.time()
        search_duration = end_search_time - start_search_time
    
        if self.enable_logging:
            print(f"\nSearch completed in {search_duration:.2f} seconds")
    
        prompt = (
            f"Context information is below.\n\n"
            f"----------------------------------------\n\n"
            f"{documents_res}\n\n"
            f"----------------------------------------\n\n"
            f"Given the context information and prior chat, answer the question below.\n\n"
            f"{question}\n"
        )
        
        if history_length:
            messages = self.messages[-history_length:] + [{"role": "user", "content": prompt}]
        else:
            messages = [{"role": "user", "content": prompt}]
        
        start_llm_time = time.time()
        response = self.llm.chat.completions.create(
            messages=messages,
            model=self.llm_model,
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        self.messages.append({"role": "user", "content": question})
        self.messages.append({"role": "assistant", "content": response.choices[0].message.content})
        
        end_llm_time = time.time()
        llm_duration = end_llm_time - start_llm_time
        
        if self.enable_logging:
            print(f"\nLLM completed in {llm_duration:.2f} seconds")
        
        return response.choices[0].message.content, documents_res

In [3]:
from rag_utils import DocLoader,StoreIndex,SearchEngine

doc_path,db_path,collection_name = 'doc','db','demo'

dl = DocLoader(path=doc_path)
si = StoreIndex(db_path=db_path,collection_name=collection_name)
si.index_from_doc_loader(dl) # si.clear() / si.delete()
se = SearchEngine(db_path=db_path, collection_name=collection_name)
rag = RAGChatbot(se,hybrid_search = True)


Loading files from doc

Processing PDFs
Skipping doc/example_doc.pdf as it has already been converted to markdown
Skipping doc/subdir/subdir_example_doc.pdf as it has already been converted to markdown

Connected to collection demo in database db for indexing.


Indexing Progress: 100%|██████████| 4/4 [00:00<00:00, 356.23it/s]


Indexing documents from doc/example_doc_pdf_converted.md.
All documents already exist in the collection.

Indexing documents from doc/example_doc.md.
All documents already exist in the collection.

Indexing documents from doc/subdir/subdir_example_doc.md.
All documents already exist in the collection.

Indexing documents from doc/subdir/subdir_example_doc_pdf_converted.md.
All documents already exist in the collection.

Indexing completed, total documents indexed: 4.

Connected to collection demo in database db for searching.

Connected to Search Engine and Language Model


In [4]:
res,doc = rag.answer("What is the meaning of learning aesthetics?")

Simple key word for text recall: Aesthetics
Key words for semantic recall: ['meaning', 'learning', 'aesthetics', 'education', 'appreciation', '意义', '学习', '美学', '教育', '欣赏']

Search completed in 4.22 seconds

LLM completed in 1.41 seconds


In [5]:
res

"Learning aesthetics involves exploring the principles and concepts related to beauty, art, and taste. It encompasses understanding how we perceive and appreciate sensory and emotional values in our surroundings. By studying aesthetics, individuals gain insights into the nature of beauty, the significance of art, and the subjective nature of taste, which can vary across different cultures and individuals. This exploration can enhance one's ability to interpret and appreciate various forms of art and the beauty in nature, fostering a deeper understanding of personal and societal values. Additionally, learning aesthetics encourages openness to new ideas and interpretations, enriching one's overall aesthetic experience."

In [6]:
doc

[{'document': '## What is Aesthetics?  \nAesthetics encompasses the study of sensory or emotional values, sometimes referred to as judgments of sentiment and taste. It examines how we perceive beauty and the principles that govern our appreciation of art and nature.',
  'path': 'doc/subdir/subdir_example_doc.md'},
 {'document': '## Conclusion  \nAesthetics is a dynamic field that continues to evolve, reflecting changes in culture, society, and technology. By exploring the nature of beauty and art, we gain insights into our values and the world around us. As we navigate an increasingly complex aesthetic landscape, it is essential to remain open to new ideas and interpretations, fostering a deeper appreciation for the beauty that surrounds us.',
  'path': 'doc/subdir/subdir_example_doc.md'},
 {'document': '# The Essence of Aesthetics  \nAesthetics, a branch of philosophy, explores the nature of beauty, art, and taste. It delves into what we find pleasing or appealing in our surroundings 

In [7]:
res,doc = rag.answer("简要介绍数学 以及和之前聊的内容的关系",history_length=2)

Simple key word for text recall: 数学
Key words for semantic recall: ['mathematics', 'introduction', 'relationship', 'content', 'discussion', '数学', '介绍', '关系', '内容', '讨论']

Search completed in 3.67 seconds

LLM completed in 1.95 seconds


In [8]:
res

'数学是研究数字、数量、形状和模式的学科，涉及抽象概念及其之间的关系。它可以分为多个分支，包括算术、代数、几何、微积分和统计等。数学不仅是解决问题的工具，也是理解周围世界的一种深刻方式。\n\n在之前的讨论中，我们提到了学习美学的概念。数学的美学体现在其结构、对称性和逻辑性上，许多人认为数学本身具有一种内在的美。通过学习数学，个体不仅能够掌握解决实际问题的技能，还能欣赏数学中的美感和深度，这与学习美学的目的相辅相成。因此，数学的学习不仅是技术性的，也是审美和哲学上的探索，帮助我们更好地理解和欣赏世界的复杂性。'

In [9]:
doc

[{'document': 'development, key concepts, and its significance in contemporary society.  \n**What is Mathematics?**  \nMathematics is the study of numbers, quantities, shapes, and patterns. It involves the exploration of  \nabstract concepts and the relationships between them. Mathematics can be divided into several\nbranches, including:  \n1. Arithmetic: The study of numbers and basic operations (addition, subtraction, multiplication,  \ndivision).',
  'path': 'doc/subdir/subdir_example_doc_pdf_converted.md'},
 {'document': '**The Beauty of Mathematics**  \nMathematics is often described as the language of the universe, a discipline that transcends cultural  \nand linguistic boundaries. It is not only a tool for solving problems but also a profound way of\nunderstanding the world around us. This article explores the essence of mathematics, its historical  \ndevelopment, key concepts, and its significance in contemporary society.  \n**What is Mathematics?**',
  'path': 'doc/subdir/subd